In [5]:
import pandas as pd
import pickle
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder

In [6]:
data=pd.read_csv('Churn_Modelling.csv')
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [7]:
# preprocess the data
data=data.drop(['RowNumber','CustomerId','Surname'], axis=1)
label_encoder_gender=LabelEncoder()
data['Gender']=label_encoder_gender.fit_transform(data['Gender'])
data.head(1)

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,0,42,2,0.0,1,1,1,101348.88,1


In [8]:
# one-hot encode geography
onehot_encoder_geo=OneHotEncoder(handle_unknown='ignore')
geo_encoded=onehot_encoder_geo.fit_transform(data[['Geography']]).toarray()
geo_encoded_df=pd.DataFrame(geo_encoded, columns=onehot_encoder_geo.get_feature_names_out(['Geography']))
geo_encoded_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0
...,...,...,...
9995,1.0,0.0,0.0
9996,1.0,0.0,0.0
9997,1.0,0.0,0.0
9998,0.0,1.0,0.0


In [9]:
data=pd.concat([data.drop('Geography', axis=1), geo_encoded_df], axis=1)
data.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [10]:
X=data.drop('EstimatedSalary', axis=1)
y=data['EstimatedSalary']

In [12]:
X_train, X_test, y_train, y_test=train_test_split(X,y, test_size=0.2, random_state=4)

In [14]:
scaler=StandardScaler()
X_train=scaler.fit_transform(X_train)
X_test=scaler.transform(X_test)

In [15]:
with open('label_encoder_gender.pkl', 'wb') as file:
    pickle.dump(label_encoder_gender, file)
with open('onehot_encoder_geo.pkl', 'wb') as file:
    pickle.dump(onehot_encoder_geo, file)
with open('scaler.pkl', 'wb') as file:
    pickle.dump(scaler, file)

## ann regression problems statement ##


In [16]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [19]:
model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(32, activation='relu'),
    Dense(1)
])

model.compile(
    optimizer='adam',
    loss='mean_absolute_error',
    metrics=['mae']
)

In [21]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6 (Dense)             (None, 64)                832       
                                                                 
 dense_7 (Dense)             (None, 32)                2080      
                                                                 
 dense_8 (Dense)             (None, 1)                 33        
                                                                 
Total params: 2945 (11.50 KB)
Trainable params: 2945 (11.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [32]:
print(model.layers[-1].activation)



<function linear at 0x0000020A0F223420>


In [22]:
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
import datetime
log_dir='regression/fit/' + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback=TensorBoard(log_dir=log_dir, histogram_freq=1)

In [23]:
tensorboard_callback


In [24]:
# early stopping
early_stopping_callback=EarlyStopping(monitor='val_loss',patience=10,restore_best_weights=True)

In [25]:
history=model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=100,
    callbacks=[early_stopping_callback, tensorboard_callback]
)

Epoch 1/100


250/250 [==============================] - 3s 4ms/step - loss: 99546.9688 - mae: 99546.9688 - val_loss: 101837.9375 - val_mae: 101837.9375
Epoch 2/100
250/250 [==============================] - 1s 3ms/step - loss: 98816.5078 - mae: 98816.5078 - val_loss: 100357.5859 - val_mae: 100357.5859
Epoch 3/100
250/250 [==============================] - 1s 3ms/step - loss: 96229.2812 - mae: 96229.2812 - val_loss: 96571.0234 - val_mae: 96571.0234
Epoch 4/100
250/250 [==============================] - 1s 3ms/step - loss: 91137.2266 - mae: 91137.2266 - val_loss: 90174.9219 - val_mae: 90174.9219
Epoch 5/100
250/250 [==============================] - 1s 3ms/step - loss: 83731.1484 - mae: 83731.1484 - val_loss: 81817.5625 - val_mae: 81817.5625
Epoch 6/100
250/250 [==============================] - 1s 3ms/step - loss: 75045.6328 - mae: 75045.6328 - val_loss: 72872.1328 - val_mae: 72872.1328
Epoch 7/100
250/250 [==============================] - 1s 3ms/step - loss: 66626.2031 - mae: 66626.2

In [27]:
%load_ext tensorboard


In [29]:
%tensorboard --logdir regressionlogs/fit

In [30]:
## evaluate model on test data
test_loss, test_mae = model.evaluate(X_test, y_test)
print(f"Test MAE : {test_mae}")

63/63 [==============================] - 0s 2ms/step - loss: 49266.4727 - mae: 49266.4727
Test MAE : 49266.47265625


In [31]:
model.save('regression_model.h5')

c:\Users\nandi\Desktop\ann-classification\venv\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
